### 【問題1】スクラッチを振り返る
```
- 重みを初期化する必要があった
- エポックのループが必要だった
- 活性化関数を作る必要があった
- optimizerを作る必要があった
- 評価関数を作る必要があった
```

### 【問題2】スクラッチとTensorFlowの対応を考える
```
全結合層: tf.keras.layers.Dense
活性化関数(Sigmoid,Tanh,Softmax,ReLU): Denseの引数 activator
optimizer(SGD,AdaGrad): model.compileのoptimizer引数
initializer(Xavier,He): glorot_normal(seed=None)
```

### 【問題3】3種類すべての目的変数を使用したIrisのモデルを作成

In [14]:
import os
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import preprocessing as prepro
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Activation

iris = datasets.load_iris()
x = iris.data
y = iris.target
# 正規化 (精度向上のため)
x = prepro.scale(x)
# ワンホットエンコーディング
y = to_categorical(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, shuffle=True)

model = Sequential()
# Denseの第一引数は隠れ層のニューロン数を、第二引数は入力層（がくの長さ、幅、花弁の長さ、幅）をタプル形式で指定
model.add(Dense(16, input_shape=(4,)))
model.add(Activation('relu'))
# ３種の分類をしたいので出力層は3を指定
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=30, batch_size=2, verbose=0)
# 評価
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy', '{:.2f}'.format(accuracy))

Accuracy 0.97


### 【問題4】House Pricesのモデルを作成

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.python import keras as K
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 6)
pd.set_option('display.max_info_rows', 6)

# データセット準備
df_base = pd.read_csv("../data/house-prices-advanced-regression-techniques/train.csv")
feature_names = ["GrLivArea", "YearBuilt"]
y_name = "SalePrice"
x = df_base.loc[:, feature_names].values
y = df_base[y_name].values

# 前処理
# 訓練データとテストデータに分ける
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=0)

# モデル準備
model = K.Sequential([
    # データの正規化（入力は１３の特徴量）
    K.layers.BatchNormalization(input_shape=(2,)),
    # １層目のニューラルネットワーク
    #   活性化関数はsoftplus
    #   kernel_regularizer正則化=>重みに制限をかける=>過学習防止
    K.layers.Dense(units=2, activation="softplus", kernel_regularizer="l1"),
    # ２層目のニューラルネットワーク
    K.layers.Dense(units=1)
])
# loss=最小二乗法  optimizer=最適化に確率的勾配降下法
model.compile(loss="mean_absolute_error", optimizer="adam")

# 学習
model.fit(x_train, y_train, epochs=10)

# 予測を行う
predicts = model.predict(x_test)

# 評価
print(model.evaluate(x_test, y_test, verbose=1))

### 【問題5】MNISTのモデルを作成


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# PrefetchDataset
mnist_dataset, mnist_info = tfds.load(name="mnist", with_info=True, as_supervised=True)
# PrefetchDataset
mnist_train, mnist_test = mnist_dataset["train"], mnist_dataset['test']

# EagerTensor
# なぜtf.castを通しているのかは現段階では不明、おそらくtf内で計算しやすいオブジェクトにしているのではなかろうか
num_valid_samples = tf.cast(0.1 * mnist_info.splits['train'].num_examples, tf.int64)
num_test_samples = tf.cast(mnist_info.splits['test'].num_examples, tf.int64)


# Feature Scaling 0~255 => 0~1 にスケーリングする
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label


# MapDataset
scaled_train_and_valid_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

# train/validにsplit
BUFFER_SIZE = 10000 # メモリ対策
shuffled_train_and_valid_data = scaled_train_and_valid_data.shuffle(BUFFER_SIZE)
valid_data = shuffled_train_and_valid_data.take(num_valid_samples)
train_data = shuffled_train_and_valid_data.skip(num_valid_samples)


# 負荷対策
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
valid_data = valid_data.batch(num_valid_samples)
test_data = test_data.batch(num_test_samples)

valid_inputs, valid_targets = next(iter(valid_data))


#
# モデルの作成
# 隠れ層: 2層
# 合計: 4層
#
input_size = 784 # 28 * 28 * 1
output_size = 10
hidden_layer_size = 50
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)), # 1列に変換
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
    tf.keras.layers.Dense(output_size, activation='softmax'),
])
model.compile(optimizer='adagrad', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 学習
result = model.fit(train_data, epochs=10, validation_data=(valid_inputs, valid_targets), verbose =2)

In [ ]:
# 評価
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

loss_list = result.history['loss']
loss_list_val = result.history['val_loss']

import matplotlib.pyplot as plt
plot_x = [i for i in range(1, len(loss_list)+1)]
plt.plot(plot_x, loss_list, label='train')
plt.plot(plot_x, loss_list_val, label='valid')
plt.legend()
plt.show()
